In [1]:
import pandas as pd

df = pd.read_csv('all_train_results_finetunedllama.csv')

In [3]:
str(df)

"                                           question_text  \\\n0          How many actions are depicted in the diagram?   \n1          How many actions are depicted in the diagram?   \n2      How many steps are there to the formation of a...   \n3           Which of these are most affected by a storm?   \n4      In a storm beach, what is exposed during the l...   \n...                                                  ...   \n19748  horizontal distance between the same points on...   \n19749  The longer the wavelength, the higher the freq...   \n19750  Space telescopes are able to gather more types...   \n19751  device that collects and focuses radio waves f...   \n19752  device that gathers and magnifies visible ligh...   \n\n                         question_image  \\\n0      question_images/erosion_6843.png   \n1      question_images/erosion_6843.png   \n2      question_images/erosion_6843.png   \n3      question_images/erosion_6845.png   \n4      question_images/erosion_6845.png   \

In [4]:
df

,question_text,question_image,answer_choices,context_image,context_image_caption,text_context,label,predicted_answer,logits
0,How many actions are depicted in the diagram?,question_images/erosion_6843.png,"a. 6, b. 4, c. 8, d. 7",teaching_images/erosion_6859.png,The diagram represents the coastal Erosion of ...,Most fossils are preserved by one of five proc...,d,EDIT,"[{'letter': 'd', 'logit': 1.4751566648483276, ..."
1,How many actions are depicted in the diagram?,question_images/erosion_6843.png,"a. 6, b. 4, c. 8, d. 7",teaching_images/erosion_6859.png,The diagram represents the coastal Erosion of ...,Most fossils are preserved by one of five proc...,d,EDIT,"[{'letter': 'd', 'logit': 1.4751566648483276, ..."
2,How many steps are there to the formation of a...,question_images/erosion_6843.png,"a. 6, b. 5, c. 10, d. 7",teaching_images/erosion_6859.png,The diagram represents the coastal Erosion of ...,"Despite these problems, there is a rich fossil...",d,conject,"[{'letter': 'c', 'logit': 1.4965717792510986, ..."
3,Which of these are most affected by a storm?,question_images/erosion_6845.png,"a. Dunes, b. Continental shelf, c. Shoreface, ...",teaching_images/parts_ocean_floor_9206.png,This diagram shows an abbreviated version of u...,Most pollutants enter the air when fossil fuel...,d,sollten,"[{'letter': 'c', 'logit': 1.524779200553894, '..."
4,"In a storm beach, what is exposed during the l...",question_images/erosion_6845.png,"a. High tide, b. Rocks, c. Upper beach, d. Dunes",teaching_images/parts_ocean_floor_9206.png,This diagram shows an abbreviated version of u...,Shores are attractive places to live and vacat...,b,áfico,"[{'letter': 'c', 'logit': 1.2372630834579468, ..."
...,...,...,...,...,...,...,...,...,...
19748,horizontal distance between the same points on...,NaN,"a. gamma ray, b. light-year, c. optical telesc...",textbook_images/wind_waves_21248.png,FIGURE 1.1,Wave speed is the distance a wave travels in a...,e,Numeric,"[{'letter': 'b', 'logit': 0.9956272840499878, ..."
19749,"The longer the wavelength, the higher the freq...",NaN,"a. true, b. false",textbook_images/wavelength_23174.png,FIGURE 1.3 Q: Of all the colors of visible lig...,Although all electromagnetic waves travel at t...,a,Massage,"[{'letter': 'a', 'logit': 0.48212572932243347,..."
19750,Space telescopes are able to gather more types...,NaN,"a. true, b. false",textbook_images/radio_waves_23046.png,FIGURE 1.2,Although all electromagnetic waves travel at t...,b,leh,"[{'letter': 'b', 'logit': 0.41581082344055176,..."
19751,device that collects and focuses radio waves f...,NaN,"a. gamma ray, b. light-year, c. optical telesc...",textbook_images/radio_waves_23045.png,FIGURE 1.1,The most important source of electromagnetic w...,g,Dialog,"[{'letter': 'b', 'logit': 1.8462666273117065, ..."


In [8]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import ast
import json

def evaluate_predictions(df):
    # Convert logits string to actual list/dict
    def get_highest_logit_letter(logits_str):
        try:
            # Convert string representation to list of dicts
            logits_list = ast.literal_eval(logits_str)
            # Find the dict with highest logit value
            max_logit_dict = max(logits_list, key=lambda x: x['logit'])
            return max_logit_dict['letter']
        except:
            return None

    # Get predictions based on highest logits
    df['predicted_label'] = df['logits'].apply(get_highest_logit_letter)
    
    # Remove rows where predictions or labels are None/NaN
    mask = df['predicted_label'].notna() & df['label'].notna()
    y_true = df.loc[mask, 'label']
    y_pred = df.loc[mask, 'predicted_label']
    print(f'GT: {y_true}\n Pred: {y_pred}\n\n')

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

    # Return confusion matrix as DataFrame
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Usage:
# df = pd.read_csv('your_file.csv')
# metrics = evaluate_predictions(df)

In [15]:
metrics = evaluate_predictions(df[:100])

GT: 0     d
1     d
2     d
3     d
4     b
     ..
95    b
96    d
97    a
98    d
99    b
Name: label, Length: 100, dtype: object
 Pred: 0     d
1     d
2     c
3     c
4     c
     ..
95    d
96    d
97    d
98    d
99    c
Name: predicted_label, Length: 100, dtype: object


Accuracy: 0.2700
F1 Score: 0.2744
Precision: 0.2976
Recall: 0.2700


/var/folders/50/ddkl_7r93pv5mq0_h58b4lv80000gn/T/ipykernel_17244/920685615.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predicted_label'] = df['logits'].apply(get_highest_logit_letter)
/Users/sakonii/Files/init/major/multimodal/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
df.iloc[1].logits

"[{'letter': 'd', 'logit': 1.4751566648483276, 'probability': 0.569893479347229}, {'letter': 'c', 'logit': 0.72099369764328, 'probability': 0.26808029413223267}, {'letter': 'a', 'logit': 0.07882039994001389, 'probability': 0.1410498172044754}, {'letter': 'b', 'logit': -1.8268929719924927, 'probability': 0.020976439118385315}]"